In [109]:
import pandas as pd

df = pd.DataFrame(columns=['Date', 'Ticker', 'Ticker Average Title Sentiment', 'Ticker Average Body Sentiment', 'Total Average Sentiment', "Next Day Ticker Price Change Direction", "5 Day Ticker Price Change Direction"])
df

,Date,Ticker,Ticker Average Title Sentiment,Ticker Average Body Sentiment,Total Average Sentiment,Next Day Ticker Price Change Direction,5 Day Ticker Price Change Direction


In [110]:
import numpy as np 
import pandas as pd 
import json
from matplotlib import pyplot as plt
with open('saps.json') as f:
  data = json.load(f)

# Displaying Subreddits 
subList = list(data.keys())
# print(subList)

In [111]:
for sub in subList:
    data[sub]['raw']['postData'] = pd.DataFrame(data = np.array(data[sub]['raw']['postData']), columns = ['Ticker','title','text','flair','unix'])
    data[sub]['raw']['postData']['rowInd'] = data[sub]['raw']['postData'].index
    # For each financial data type
    for finType in ['inter','intra']:
    
        # Get a list of post keys
        keyList = list(data[sub]['raw'][finType].keys())
    
        # For each keys, get the financial data for that post
        for key in keyList:
            data[sub]['raw'][finType][key] = np.array(data[sub]['raw'][finType][key])
            data[sub]['raw'][finType][key] = pd.Series(data[sub]['raw'][finType][key][:,1], index = data[sub]['raw'][finType][key][:,0])

In [112]:

# keys = data['Stocks']['md']['inter']['keys']
# sample = data['Stocks']['raw']['inter'][keys[1100]]
# sample_df = pd.DataFrame(data = sample).reset_index()
# sample_df.columns = ['unix','percentChange']
# sample_df['date'] = pd.to_datetime(sample_df['unix'],unit='s', utc=True)
# sample_df['date'] = sample_df['date'].dt.date
# # sample_df = sample_df.set_index('date')
# # sample_df['cumulativeChange'] = sample_df.groupby('date')['percentChange'].cumsum()
# display(sample_df.tail(20))
# # data['Stocks']['raw']['postData'].loc[0]
# display(data['Stocks']['raw']['postData'].loc[1100])
# sample_ticker = data['Stocks']['raw']['postData']['ticker'][1100]
# post_unix = data['Stocks']['raw']['postData']['unix'][1100]
# pd.to_datetime(post_unix,unit='s', utc=True).tz_convert('US/Eastern')

In [113]:
postData_df = pd.DataFrame()
for sub in subList:
    subData_df = data[sub]['raw']['postData']
    subData_df['subreddit'] = sub
    postData_df = pd.concat([postData_df, subData_df], axis = 0)

postData_df['Date'] = pd.to_datetime(postData_df['unix'], unit = 's', utc = True)
postData_df['Date'] = postData_df['Date'].dt.tz_convert('US/Eastern').dt.date
postData_df = postData_df.drop(columns = ['unix'])
postData_df.reset_index(drop = True, inplace = True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20620\4096438045.py:7: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  postData_df['Date'] = pd.to_datetime(postData_df['unix'], unit = 's', utc = True)


In [114]:
postData_df["Sentiment"] = np.random.uniform(-1,1, size=(len(postData_df),))
postData_df['Market Average Sentiment'] = postData_df.groupby(['Date'])['Sentiment'].transform('mean')
# postData_df['Daily Ticker Title Average Sentiment'] = postData_df.groupby(['Date', 'ticker'])['Title Sentiment'].transform('mean')
# postData_df['Daily Ticker Body Average Sentiment'] = postData_df.groupby(['Date', 'ticker'])['Body Sentiment'].transform('mean')
# postData_df

In [127]:
import yfinance as yf

model_input = postData_df.groupby(['Date', 'Ticker', 'Market Average Sentiment']).agg({'Sentiment': 'mean'}).reset_index()
start_date = model_input['Date'].min() - pd.Timedelta(days=5)
end_date = model_input['Date'].max() + pd.Timedelta(days=10)

# # Download all data at once
# tickers = model_input['ticker'].unique().tolist()
# ticker_string = " ".join(tickers)  # Convert list of tickers into a space-separated string
# data = yf.download(ticker_string, start=start_date, end=end_date)

In [64]:
data.to_pickle("stock_data.pkl")

In [118]:
data = pd.read_pickle("stock_data.pkl")

In [99]:
all_dates = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='D'))
all_dates.reset_index(inplace=True)
all_dates.columns = ['Date']
all_dates['Date'] = all_dates['Date'].dt.date

In [119]:
next_day_pct_change = data['Close'].pct_change().__deepcopy__()
next_day_pct_change.dropna(axis = 1, how='all', inplace = True)
next_day_pct_change = next_day_pct_change.shift(-1)
next_day_pct_change.dropna(axis = 0, how='all', inplace = True)
next_day_pct_change.reset_index(inplace = True)
next_day_pct_change['Date'] = pd.to_datetime(next_day_pct_change['Date']).dt.date
next_day_pct_change = next_day_pct_change.merge(all_dates, on = 'Date', how = 'right').fillna(method='bfill')
next_day_pct_change = next_day_pct_change.melt(id_vars = 'Date', var_name = 'Ticker', value_name = 'Next Day Price Change')
next_day_pct_change.dropna(subset=['Next Day Price Change'], inplace=True)
next_day_pct_change['Next Day Price Change Direction'] = (next_day_pct_change['Next Day Price Change'] > 0).astype(int)
# display(next_day_pct_change)

In [126]:
next_5d_pct_change = data['Close'].pct_change(5).__deepcopy__()
next_5d_pct_change.dropna(axis = 1, how='all', inplace = True)
next_5d_pct_change = next_5d_pct_change.shift(-5)
next_5d_pct_change.dropna(axis = 0, how='all', inplace = True)
next_5d_pct_change.reset_index(inplace = True)
next_5d_pct_change['Date'] = pd.to_datetime(next_5d_pct_change['Date']).dt.date
next_5d_pct_change = next_5d_pct_change.merge(all_dates, on = 'Date', how = 'right').fillna(method='bfill')
next_5d_pct_change = next_5d_pct_change.melt(id_vars = 'Date', var_name = 'Ticker', value_name = 'Next 5 Day Price Change')
next_5d_pct_change.dropna(subset=['Next 5 Day Price Change'], inplace=True)
next_5d_pct_change['Next 5 Day Price Change Direction'] = (next_5d_pct_change['Next 5 Day Price Change'] > 0).astype(int)
next_5d_pct_change['Date'] = pd.to_datetime(next_5d_pct_change['Date']).dt.date
# display(next_5d_pct_change)

In [128]:
# model_input['Next Day Ticker Price Change Direction'] = np.nan
# model_input['5 Day Ticker Price Change Direction'] = np.nan
model_input = model_input.merge(next_day_pct_change, how='left', on=['Date', 'Ticker'])
model_input = model_input.merge(next_5d_pct_change, how='left', on=['Date', 'Ticker'])
model_input.drop(columns=['Next Day Price Change', 'Next 5 Day Price Change'], inplace=True)
model_input.dropna(subset=['Next Day Price Change Direction', 'Next 5 Day Price Change Direction'], inplace=True)

# for day in tqdm.tqdm(model_input['Date'].unique(), desc='Processing Days'):
#     day_stock_data_loc = stock_data[stock_data['Date'] >= day].index[0]
#     if stock_data.loc[day_stock_data_loc, 'Date'] != day:
#         day_stock_data_loc -= 1
#     next_day_stock_data = stock_data.loc[day_stock_data_loc + 1]
#     five_days_stock_data = stock_data.loc[day_stock_data_loc + 1:day_stock_data_loc + 5]
#     five_days_stock_data.iloc[:, 1:] = five_days_stock_data.iloc[:, 1:].apply(lambda x: x + 1)
#     five_days_stock_data.iloc[:, 1:] = five_days_stock_data.iloc[:, 1:].cumprod() - 1
#     day_model_input = model_input[model_input['Date'] == day]
#     for ticker in day_model_input['ticker'].unique():
#         model_input.loc[model_input['ticker'] == ticker, 'Next Day Ticker Price Change Direction'] = next_day_stock_data[ticker] > 0
#         model_input.loc[model_input['ticker'] == ticker, '5 Day Ticker Price Change Direction'] = five_days_stock_data[ticker].iloc[-1] > 0

# model_input.dropna(subset=['Next Day Ticker Price Change Direction', '5 Day Ticker Price Change Direction'], inplace=True)
display(model_input)

,Date,Ticker,Market Average Sentiment,Sentiment,Next Day Price Change Direction,Next 5 Day Price Change Direction
0,2017-02-19,MOMO,0.963644,0.963644,0.0,0.0
6,2017-03-30,GAIN,0.453181,0.949398,1.0,0.0
7,2017-03-30,IDXG,0.453181,-0.043035,0.0,0.0
8,2017-04-04,NSPR,0.313778,0.313778,0.0,0.0
9,2017-04-07,TOPS,-0.628831,-0.628831,1.0,1.0
...,...,...,...,...,...,...
29082,2020-09-23,NKLA,-0.147264,0.993446,0.0,0.0
29083,2020-09-23,QYLD,-0.147264,0.015138,1.0,1.0
29084,2020-09-23,SPY,-0.147264,-0.442850,1.0,1.0
29085,2020-09-23,TSLA,-0.147264,-0.570425,1.0,1.0


In [129]:
spy_data = yf.download('SPY', start=start_date, end=end_date)
spy_data = spy_data['Close'].pct_change().reset_index()
full_date_range = pd.DataFrame({'Date': pd.date_range(start=start_date, end=end_date)})
full_date_range["Date"] = full_date_range["Date"].dt.date
spy_data['Date'] = pd.to_datetime(spy_data['Date']).dt.date
spy_data = full_date_range.merge(spy_data, on='Date', how='left')
spy_data.fillna(method='ffill', inplace=True)
spy_data = spy_data.rename(columns={'Close': 'SPY'})
model_input = model_input.merge(spy_data, on='Date', how='left')
model_input.dropna(subset=['SPY'], inplace=True)
display(model_input)
model_input.to_csv('model_input.csv', index=False)

[*********************100%%**********************]  1 of 1 completed


,Date,Ticker,Market Average Sentiment,Sentiment,Next Day Price Change Direction,Next 5 Day Price Change Direction,SPY
0,2017-02-19,MOMO,0.963644,0.963644,0.0,0.0,0.001576
1,2017-03-30,GAIN,0.453181,0.949398,1.0,0.0,0.003184
2,2017-03-30,IDXG,0.453181,-0.043035,0.0,0.0,0.003184
3,2017-04-04,NSPR,0.313778,0.313778,0.0,0.0,0.000637
4,2017-04-07,TOPS,-0.628831,-0.628831,1.0,1.0,-0.001019
...,...,...,...,...,...,...,...
24057,2020-09-23,NKLA,-0.147264,0.993446,0.0,0.0,-0.023191
24058,2020-09-23,QYLD,-0.147264,0.015138,1.0,1.0,-0.023191
24059,2020-09-23,SPY,-0.147264,-0.442850,1.0,1.0,-0.023191
24060,2020-09-23,TSLA,-0.147264,-0.570425,1.0,1.0,-0.023191
